In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/Kannada-MNIST/sample_submission.csv
/kaggle/input/Kannada-MNIST/test.csv
/kaggle/input/Kannada-MNIST/Dig-MNIST.csv
/kaggle/input/Kannada-MNIST/train.csv


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
BATCH_SIZE = 32
EPOCHS = 100
IMG_W = 28
IMG_H = 28

In [4]:
train = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test = pd.read_csv('/kaggle/input//Kannada-MNIST/test.csv')
digi = pd.read_csv('/kaggle/input//Kannada-MNIST/Dig-MNIST.csv')

In [5]:
def process_data(data, dims):
    X = data.iloc[:,1:].to_numpy(dtype="float32")
    y = data['label'].to_numpy()
    
    X = X/255
    X = X.reshape(-1, dims[0], dims[1], 1)
    y = to_categorical(y)
    
    print("X.shape: {}, y.shape: {}".format(X.shape, y.shape))
    
    return X,y

In [6]:
#train.head()
#test.head()
digi.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X_val, y_val = process_data(digi, dims=(IMG_W, IMG_H))

X.shape: (10240, 28, 28, 1), y.shape: (10240, 10)


In [8]:
X_train, y_train = process_data(train, dims=(IMG_W, IMG_H))

X.shape: (60000, 28, 28, 1), y.shape: (60000, 10)


In [9]:
model = Sequential()

model.add(Conv2D(128,(3,3),activation='relu', input_shape=(28,28,1)))
# model.add(BatchNormalization(momentum=0.2, gamma_initializer='uniform'))
# model.add(MaxPooling2D())
model.add(Conv2D(128,(3,3),activation='relu'))
# model.add(BatchNormalization(momentum=0.1, gamma_initializer='uniform'))
model.add(MaxPool2D((2,2)))
# model.add(Dropout(0.2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
# model.add(BatchNormalization(momentum=0.1, gamma_initializer='uniform'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization(momentum=0.1, gamma_initializer='uniform'))
model.add(MaxPool2D((2,2)))
# model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])

In [10]:
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), verbose=2, callbacks=[EarlyStopping(monitor='val_acc', patience=5)])
# model.fit(X_train, y_train, epochs=5, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), verbose=2)

Train on 60000 samples, validate on 10240 samples
Epoch 1/100
60000/60000 - 17s - loss: 0.0944 - acc: 0.9707 - val_loss: 1.6702 - val_acc: 0.6729
Epoch 2/100
60000/60000 - 12s - loss: 0.0391 - acc: 0.9881 - val_loss: 1.8674 - val_acc: 0.6340
Epoch 3/100
60000/60000 - 12s - loss: 0.0321 - acc: 0.9903 - val_loss: 1.4496 - val_acc: 0.7091
Epoch 4/100
60000/60000 - 13s - loss: 0.0253 - acc: 0.9921 - val_loss: 1.2671 - val_acc: 0.7498
Epoch 5/100
60000/60000 - 12s - loss: 0.0206 - acc: 0.9936 - val_loss: 1.2620 - val_acc: 0.7563
Epoch 6/100
60000/60000 - 12s - loss: 0.0184 - acc: 0.9941 - val_loss: 1.4678 - val_acc: 0.7312
Epoch 7/100
60000/60000 - 12s - loss: 0.0167 - acc: 0.9945 - val_loss: 1.8085 - val_acc: 0.7082
Epoch 8/100
60000/60000 - 12s - loss: 0.0119 - acc: 0.9963 - val_loss: 1.2585 - val_acc: 0.7826
Epoch 9/100
60000/60000 - 13s - loss: 0.0139 - acc: 0.9958 - val_loss: 1.3428 - val_acc: 0.7594
Epoch 10/100
60000/60000 - 12s - loss: 0.0094 - acc: 0.9970 - val_loss: 1.6209 - val_a

In [11]:
X_test = test.iloc[:,1:].to_numpy(dtype="float32")

X_test = X_test.reshape(-1,IMG_W, IMG_W, 1)
X_test.shape

(5000, 28, 28, 1)

In [12]:
predictions_new = model.predict(X_test)
result = [np.argmax(pred) for pred in predictions_new]

# Save test predictions to file
output = pd.DataFrame({"id": test.id, "label": result})
output.to_csv("submission.csv", index=False)